# Property Package Introduction

The properties of water in WaterTAP models are defined by the [property models](https://watertap.readthedocs.io/en/stable/technical_reference/property_models/index.html). 

In WaterTAP unit models, the property models representing the inlet and outlet stream are sub-models of the main unit model called _state blocks_. State blocks describe physicochemical properties of a given stream.

The creation of state blocks is done automatically as part of the build for any unit model.

This tutorial will cover the basics of interacting with property packages and state blocks in WaterTAP. We will do this via creation of a feed block with the `Feed` model. This model is typically the starting point for any WaterTAP flowsheet.


### Required imports from Pyomo, IDAES, and WaterTAP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from pyomo.environ import ConcreteModel, value, units as pyunits

from idaes.core import FlowsheetBlock
from idaes.models.unit_models import Feed
from idaes.core.util.model_statistics import degrees_of_freedom

from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock
from watertap.property_models.NaCl_T_dep_prop_pack import (
    NaClParameterBlock as NaClParameterBlock_Tdep,
)
from watertap.core.solvers import get_solver

### Create model, flowsheet, and property model


In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

### Add property package

The property model contains all the instructions for creating state blocks on unit models.

<center><img src="graphics/sw_properties.png" width="800" /></center>

In [ ]:
m.fs.properties = SeawaterParameterBlock()

<center><img src="graphics/prop_pack1.png" width="400" /></center>

### Add `Feed` model to flowsheet

For the seawater property model (and many other WaterTAP property models), the state variables are:
- pressure
- temperature
- mass flow rate of components 

In [ ]:
m.fs.feed = Feed(property_package=m.fs.properties)
m.fs.feed.report()
# m.fs.feed.properties[0].display()

<center><img src="graphics/prop_pack2.png" width="400"/></center>

### Creating other properties

In [ ]:
# Touch conc_mass_phase_comp to create it and any associated properties
m.fs.feed.properties[0].conc_mass_phase_comp
m.fs.feed.properties[0].display()

<center><img src="graphics/prop_pack3.png" width="400" /></center>

### Setting the state variables

For all the properties we have created to be constrained, we only need to fix the values for the state variables.


In [ ]:
# Fix mass flow of water
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)

In [ ]:
# Fix mass flow of TDS
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)

In [ ]:
# Fix temperature
m.fs.feed.properties[0].temperature.fix(273 + 25)

In [ ]:
# Fix pressure
m.fs.feed.properties[0].pressure.fix(101325)

In [ ]:
# Check degrees of freedom are zero
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")


<center><img src="graphics/prop_pack4.png" width="400" /></center>

### Initialize and solve to determine the concentration

In [ ]:
m.fs.feed.initialize()

solver = get_solver()
results = solver.solve(m)
print(f"Solve termination {results.solver.termination_condition}")
print(
    f"Concentration of TDS: {value(m.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'TDS']):.2f} g/L"
)

### Repeat with osmotic pressure

In [ ]:
# Create model, flowsheet, and property package
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

# Add feed model
m.fs.feed = Feed(property_package=m.fs.properties)

In [ ]:
# Touch pressure_osm_phase to create it
m.fs.feed.properties[0].pressure_osm_phase

In [ ]:
# Fix mass flow of water
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)

In [ ]:
# Fix mass flow of TDS
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)

In [ ]:
# Fix temperature
m.fs.feed.properties[0].temperature.fix(273 + 25)

In [ ]:
# Fix pressure
m.fs.feed.properties[0].pressure.fix(101325)

In [ ]:
# Check degrees of freedom are zero
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")
assert degrees_of_freedom(m) == 0

In [ ]:
# Initialize and solve model
m.fs.feed.initialize()
results = solver.solve(m)

print(f"Solve termination {results.solver.termination_condition}")
print(
    f"Osmotic Pressure: {value(m.fs.feed.properties[0].pressure_osm_phase['Liq']):.2f} Pa"
)

### Solve for other variables using a specified osmotic pressure


In [ ]:
# Touch conc_mass_phase_comp
m.fs.feed.properties[0].conc_mass_phase_comp

# Fix osmotic pressure to desired value
osmotic = 12 * pyunits.bar
m.fs.feed.properties[0].pressure_osm_phase["Liq"].fix(osmotic)

# Unfix the mass flow of TDS
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].unfix()
print(f"Degrees of freedom: {degrees_of_freedom(m)}\n")

results = solver.solve(m)
print(f"Solve termination {results.solver.termination_condition}")
print(
    f"TDS Concentration: {value(m.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'TDS']):.4f} g/L"
)

# Try It Yourself #1 

### Create a feed block with the seawater property package to estimate the osmotic pressure and salinity for the following conditions:

- Mass flow water = 3.3 kg/s
- Mass flow TDS = 0.55 kg/s
- Temperature = 20 C
- Pressure = 1 bar

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

# Continue Flowsheet Setup Below
# ================================

# Create feed block with seawater parameter block

# Touch the necessary variables to create them in the model

# Fix the state variables: mass flow of water, mass flow of TDS, temperature, pressure





# assert degrees_of_freedom(m) == 0
# results = solver.solve(m)
# print(f"Solve termination {results.solver.termination_condition}")
# print(
#     f"Osmotic Pressure (Pa): {value(m.fs.feed.properties[0].pressure_osm_phase['Liq']/1e5):.2f} Pa"
# )
# print(
#     f"Concentration (kg/m3): {value(m.fs.feed.properties[0].conc_mass_phase_comp['Liq', 'TDS']):.2f} g/L"
# )

# Try It Yourself #2

### Create a feed block with the seawater property package to estimate the osmotic pressure for the following conditions:

- TDS concentration = 55 g/L
- Volumetric flow rate = 1 m3/s
- Temperature = 20 C
- Pressure = 1 bar

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = SeawaterParameterBlock()

# Continue Flowsheet Setup Below
# ================================

# Create feed block with seawater parameter block

# Touch the necessary variables to create them in the model


# Fix the volumetric flow rate, concentration, temperature, and pressure
# as specified in the problem statement





# assert degrees_of_freedom(m) == 0
# results = solver.solve(m)
# print(f"Solve termination {results.solver.termination_condition}")
# print(
#     f"Osmotic Pressure (Pa): {value(m.fs.feed.properties[0].pressure_osm_phase['Liq']):.2f} Pa"
# )

### Compare seawater and NaCl property packages

Low salinity range: 5-70 g/L

In [ ]:
# =======================================================================================
# Build the model for seawater property package


def build_seawater_prop_model():
    """
    Create feed model using seawater property package
    """
    m1 = ConcreteModel()
    m1.fs = FlowsheetBlock(dynamic=False)
    m1.fs.properties = SeawaterParameterBlock()
    m1.fs.feed = Feed(property_package=m1.fs.properties)
    m1.fs.feed.properties[0].pressure_osm_phase
    m1.fs.feed.properties[0].conc_mass_phase_comp
    m1.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
    m1.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].fix(0.035)
    m1.fs.feed.properties[0].temperature.fix(273 + 25)
    m1.fs.feed.properties[0].pressure.fix(101325)

    print(f"Degrees of freedom: {degrees_of_freedom(m1)}\n")

    # Initialize and solve for the initial conditions
    m1.fs.feed.initialize()
    results = solver.solve(m1)
    print(f"Solve termination {results.solver.termination_condition}")
    # Unfix TDS mass flow
    m1.fs.feed.properties[0].flow_mass_phase_comp["Liq", "TDS"].unfix()

    return m1


m1 = build_seawater_prop_model()


# =======================================================================================
# Build the model for NaCl property package
def build_NaCl_prop_model():
    """
    Create feed model using NaCl property package
    """
    m2 = ConcreteModel()
    m2.fs = FlowsheetBlock(dynamic=False)
    m2.fs.properties = NaClParameterBlock()
    m2.fs.feed = Feed(property_package=m2.fs.properties)
    m2.fs.feed.properties[0].pressure_osm_phase
    m2.fs.feed.properties[0].conc_mass_phase_comp
    m2.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
    m2.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].fix(0.035)
    m2.fs.feed.properties[0].temperature.fix(273 + 25)
    m2.fs.feed.properties[0].pressure.fix(101325)

    print(f"Degrees of freedom: {degrees_of_freedom(m2)}\n")

    # Initialize and solve for the initial conditions
    m2.fs.feed.initialize()
    results = solver.solve(m2)
    print(f"Solve termination {results.solver.termination_condition}")
    # Unfix NaCl mass flow
    m2.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].unfix()

    return m2


m2 = build_NaCl_prop_model()

# =======================================================================================
# Define a range of concentrations
low_concs = np.linspace(5, 70, 25)  # g/L
# Create empty lists to store osmotic pressures for each model
p_osm1_low = []
p_osm2_low = []

# Solves for osmotic pressure over a range of concentrations
for c in low_concs:
    m1.fs.feed.properties[0].conc_mass_phase_comp["Liq", "TDS"].fix(c)
    m2.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"].fix(c)
    results1 = solver.solve(m1)
    results2 = solver.solve(m2)
    p_osm1_low.append(
        value(
            pyunits.convert(
                m1.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )
    p_osm2_low.append(
        value(
            pyunits.convert(
                m2.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )


# =======================================================================================
# Plot the results

fig, ax = plt.subplots()

ax.scatter(low_concs, p_osm1_low, color="blue", label="Seawater Property Package")
ax.scatter(low_concs, p_osm2_low, color="red", label="NaCl Property Package")
ax.set_xlabel("Concentration (g/L)")
ax.set_ylabel("Osmotic Pressure (bar)")
ax.set_title("Osmotic Pressure vs Concentration (Low Range)")
ax.legend()

### Compare seawater and NaCl property packages

High salinity range: 70-250 g/L

In [ ]:
m1 = build_seawater_prop_model()
m2 = build_NaCl_prop_model()

# =======================================================================================
# Define a range of concentrations
high_concs = np.linspace(70, 250, 25)  # g/L
# Create empty lists to store osmotic pressures for each model
p_osm1_high = []
p_osm2_high = []

# Solves for osmotic pressure over a range of concentrations
for c in high_concs:
    m1.fs.feed.properties[0].conc_mass_phase_comp["Liq", "TDS"].fix(c)
    m2.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"].fix(c)
    results1 = solver.solve(m1)
    results2 = solver.solve(m2)
    p_osm1_high.append(
        value(
            pyunits.convert(
                m1.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )
    p_osm2_high.append(
        value(
            pyunits.convert(
                m2.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )

# =======================================================================================
# Plot the results

fig, ax = plt.subplots()

ax.scatter(high_concs, p_osm1_high, color="blue", label="Seawater Property Package")
ax.scatter(high_concs, p_osm2_high, color="red", label="NaCl Property Package")
ax.set_xlabel("Concentration (g/L)")
ax.set_ylabel("Osmotic Pressure (bar)")
ax.set_title("Osmotic Pressure vs Concentration (High Range)")
ax.legend()

### Compare NaCl property temperature dependence


In [ ]:
# =======================================================================================
# Build the model for NaCl property package


def build_NaCl_prop_model2():
    """
    Create feed model using NaCl property package for assessment of temperature dependence
    """
    m3 = ConcreteModel()
    m3.fs = FlowsheetBlock(dynamic=False)
    m3.fs.properties = NaClParameterBlock()
    m3.fs.feed = Feed(property_package=m3.fs.properties)
    m3.fs.feed.properties[0].pressure_osm_phase
    m3.fs.feed.properties[0].conc_mass_phase_comp
    m3.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
    m3.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].fix(0.035)
    m3.fs.feed.properties[0].temperature.fix(273 + 25)
    m3.fs.feed.properties[0].pressure.fix(101325)

    print(f"Degrees of freedom: {degrees_of_freedom(m3)}\n")

    # Initialize and solve for the initial conditions
    m3.fs.feed.initialize()
    results = solver.solve(m3)
    print(f"Solve termination {results.solver.termination_condition}")
    # Unfix temperature
    m3.fs.feed.properties[0].temperature.unfix()

    return m3


m3 = build_NaCl_prop_model2()

# =======================================================================================
# Build the model for NaCl property package with temperature dependence


def build_NaCl_T_dep_prop_model():
    """
    Create feed model using NaCl property package with temperature dependence
    """
    m4 = ConcreteModel()
    m4.fs = FlowsheetBlock(dynamic=False)
    m4.fs.properties = NaClParameterBlock_Tdep()
    m4.fs.feed = Feed(property_package=m4.fs.properties)
    m4.fs.feed.properties[0].pressure_osm_phase
    m4.fs.feed.properties[0].conc_mass_phase_comp
    m4.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(0.965)
    m4.fs.feed.properties[0].flow_mass_phase_comp["Liq", "NaCl"].fix(0.035)
    m4.fs.feed.properties[0].temperature.fix(273 + 25)
    m4.fs.feed.properties[0].pressure.fix(101325)

    print(f"Degrees of freedom: {degrees_of_freedom(m4)}\n")

    # Initialize and solve for the initial conditions
    m4.fs.feed.initialize()
    results = solver.solve(m4)
    print(f"Solve termination {results.solver.termination_condition}")
    # Unfix temperature
    m4.fs.feed.properties[0].temperature.unfix()

    return m4


m4 = build_NaCl_T_dep_prop_model()


# =======================================================================================
# Define a range of temperatures
temps_C = np.linspace(1, 99, 20)  # °C
# Ensure 25 °C is included
temps_C = np.append(temps_C, 25)
# Create empty lists to store osmotic pressures and concentrations for each model
c3 = []
c4 = []
p_osm3 = []
p_osm4 = []

# Solves for osmotic pressure across a range of concentrations
for t in temps_C:
    m3.fs.feed.properties[0].temperature.fix(273 + t)
    m4.fs.feed.properties[0].temperature.fix(273 + t)
    results3 = solver.solve(m3)
    results4 = solver.solve(m4)
    c3.append(value(m3.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"]))
    c4.append(value(m4.fs.feed.properties[0].conc_mass_phase_comp["Liq", "NaCl"]))
    p_osm3.append(
        value(
            pyunits.convert(
                m3.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )
    p_osm4.append(
        value(
            pyunits.convert(
                m4.fs.feed.properties[0].pressure_osm_phase["Liq"], to_units=pyunits.bar
            )
        )
    )
    if t == 25:
        # Results at 25 °C should be nearly identical for both property models
        print(
            f"\nAt 25 °C, NaCl Property Package: Osmotic Pressure = {p_osm3[-1]:.2f} bar, Concentration = {c3[-1]:.2f} g/L"
        )
        print(
            f"At 25 °C, NaCl T-dependent Property Package: Osmotic Pressure = {p_osm4[-1]:.2f} bar, Concentration = {c4[-1]:.2f} g/L"
        )

# =======================================================================================
# Plot the osmotic pressure results
fig, ax = plt.subplots()
ax.scatter(temps_C, p_osm3, color="red", label="NaCl Property Package")
ax.scatter(temps_C, p_osm4, color="green", label="NaCl T-dependent Property Package")
ax.vlines(
    25,
    ymin=min(p_osm3 + p_osm4),
    ymax=max(p_osm3 + p_osm4),
    colors="gray",
    linestyles="dashed",
)
ax.set_xlabel("Temperature (°C)")
ax.set_ylabel("Osmotic Pressure (bar)")
ax.set_title("Osmotic Pressure vs Temperature")
ax.legend()

# Plot the concentration results
fig, ax = plt.subplots()
ax.scatter(temps_C, c3, color="red", label="NaCl Property Package")
ax.scatter(temps_C, c4, color="green", label="NaCl T-dependent Property Package")
ax.vlines(25, ymin=min(c3 + c4), ymax=max(c3 + c4), colors="gray", linestyles="dashed")
ax.set_xlabel("Temperature (°C)")
ax.set_ylabel("Concentration (g/L)")
ax.set_title("Concentration vs Temperature")
ax.legend()